In [271]:
import configparser
from pathlib import Path
import pandas as pd
import pandas.io.sql as pandasql
from psycopg2.extras import execute_values
from psycopg2 import connect
import numpy
import matplotlib as mpl
from IPython.display import HTML, display
import rick
import numpy as np
import matplotlib.pyplot as plt
from setuptools import setup, find_packages
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [279]:
merge_set = pd.read_sql('''select rank, start_vid, end_vid, length, link_set, new_start_vid, new_end_vid, new_length, new_link_set, st_hausdorffdistance  from test_route_ranks ''',con)
merge_set['length'].values.astype(int)
merge_set['new_length'].values.astype(int)

array([ 172,  294,   82,  172,  294,  518,  172,  648,  779,  649, 1076,
        685,  726,   82,  649, 1064, 1209, 1064, 1076,  545, 1064, 1209,
        998,  647,  958,  649, 1076, 1064, 1076, 1157, 1161,  545,  530,
        545,  998, 1034, 1043,  647, 1047,  499,  757,  332,  647, 1034,
       1048,  446,  757,  734,  953,  518,  656,  280,  467,  648,  849,
        777,  778,  866,  341,  467,  649,  866,  276,  467,  999, 1164,
        656,  950,  240,  656,  341,  467,  240,  656,  240,  872,  658,
        779,  229,  280,  649,  820,  230,  590,  685, 1116,  229,  586,
        366,  645,  366,  645,  451,  798,  590, 1062,  230,  276,  982,
        983,  366,  798,  929, 1051,  533,  798,  691, 1157,  586, 1066,
        361,  691,  361,  691,  712, 1062,  790,  824,  714, 1066,  105,
        106,  108,  532,  105,  108,  310,  712,  105,  328,  306,  714,
        310,  501,  306,  499,  642,  760,  829, 1051,  790,  824, 1051,
       1205,  915, 1018, 1051, 1205,  693,  725, 10

In [286]:
link_used = []
new_sets = []
rows = []
unique_set = merge_set[['start_vid', 'end_vid']].drop_duplicates()

# loop through unique sets of intersection
# !!this will be ordered by priority and length eventually!!
for index, key in unique_set.reset_index(drop=True).iterrows(): 
    
    start_vid = unique_set['start_vid'].iloc[index]
    end_vid = unique_set['end_vid'].iloc[index]
    #print(start_vid, end_vid)
    prepare_set = merge_set[(merge_set['start_vid'] == start_vid) & (merge_set['end_vid'] == end_vid)]
    link_set = prepare_set['link_set'].iloc[0]

    # Check if this link_set was used to match another set before
    # if not, selection best and merge
    if link_used == [] or np.any([link in link_used for link in link_set]) == False:
        #print('not used')
        # if there are more than one rank and length is less than 100 or something 
        
        if prepare_set['length'].iloc[0] < 200:    
            # if there are two possibility for this intersection set
            # then compare which one is better

            if prepare_set['rank'].shape[0] == 2: 
                rank1=prepare_set[(merge_set['rank']==1)]
                rank2=prepare_set[(merge_set['rank']==2)]

                # find hausdorffdistance difference and length difference
                hausorffdiff = abs(rank1['st_hausdorffdistance'].iloc[0]/rank2['st_hausdorffdistance'].iloc[0])
                lengthdiff = rank1['new_length'].iloc[0] - rank2['new_length'].iloc[0]

                # make selection based on hausdorffdistance difference and length difference
                # could try and play with the params a little bit
                if hausorffdiff < 5 and lengthdiff > 20:
                    select_set = 2
                else:
                    select_set = 1

                selection = merge_set[(merge_set['rank']==select_set)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                
                
                # find links that were used to merge, append it to another list
                # also have to find out if it was using a set of link that was used before
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                
                if np.any([link in additional_links for link in link_used]) == False:
                    link_used.append(additional_links)
                    new_sets.append(new_set)
                    
            # if there is only one possibility
            # see if that possibility is viable 
            elif prepare_set['rank'].shape[0] == 1:
                selection = merge_set[(merge_set['rank']==1)&(merge_set['start_vid']==start_vid)&(merge_set['end_vid']==end_vid)]
                old_links = selection['link_set'].iloc[0]
                new_links = selection['new_link_set'].iloc[0]
                additional_links = list(set(new_links).difference(old_links))
                new_set = np.array([selection['new_start_vid'].iloc[0], selection['new_end_vid'].iloc[0], selection['new_length'].iloc[0], selection['new_link_set'].iloc[0]])
                if np.any([link in additional_links for link in link_used]) == False:
                    link_used.append(additional_links)
                    new_sets.append(new_set)
                else:
                    print('was used before')
                
        
        # if the set length is longer than 100 then 
        # check to see if it was used to merge 
        # if yes dont append, if not append
        else:  
            if np.any([link in prepare_set['link_set'].iloc[0] for link in link_used]) != False:
                print('was used before')
            else:
                new_set = np.array([prepare_set['start_vid'].iloc[0], prepare_set['end_vid'].iloc[0], prepare_set['length'].iloc[0], prepare_set['link_set'].iloc[0]])
                new_sets.append(new_set)
                link_used.append(prepare_set['link_set'].iloc[0])
                
    # if link_set as used, continue and dont append
    elif np.any([link in link_used for link in link_set]) == True:
        print('used')

for i in range(len(new_sets)):
    item = new_sets[i]
    length = item[2].astype(float)

    row = (item[0].astype(float), item[1].astype(float), length, item[3])
    #print(type(item[0].astype(int)),type(item[1]),type(item[2]),type(item[3]) )
    rows.append(row)
        
sql = '''insert into congestion.test_shortseg(start_vid, end_vid, length, link_set) VALUES %s'''    
with con:
    with con.cursor() as cur:
        execute_values(cur, sql, rows)    

        

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:26: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/etc/jupyterhub/.venv/lib/python3.5/site-packages/ipykernel_launcher.py:27: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [265]:
for i in range(len(new_sets)):
    item = new_sets[i]
    row = (item[0], item[1], item[2], item[3])
    rows.append(row)


[(862292856, 30362990, 172.271723526887, [7980069201, 9456288911, 9456288901, 295895041]), (30362987, 30363018, 82.9219338709931, [7980069200, 7980069210]), (30362987, 30362979, 294.675112162196, [9456288911, 9456288901, 295895041, 7546081061, 10558928811, 10708227301, 10708227291]), (30362990, 862292856, 172.271723526887, [295895040, 9456288900, 9456288910, 7980069200]), (30362990, 30414528, 632.851216206641, [295895030, 295894770, 10566913110, 10566913120, 8266390160, 8266390170, 9455932580, 9455932590, 7994232670, 7994232680]), (30363018, 30363091, 592.403911602748, [295893130, 295892440, 295892050, 295891530, 295891250, 12056277700, 12056277710, 7545625650, 9488531850, 9488531860, 7545625620, 10558927980, 10558927990, 9456275600]), (30363018, 30414576, 627.93567120371, [7980069240, 7980069250, 295892560, 10566913020, 10566913030, 10566912930, 10566912940, 11590148400, 11590148410, 295891890, 295891870, 295891700]), (30363018, 30362987, 82.9219338709931, [7980069211, 7980069201]), (